In [77]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [78]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text


In [79]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [80]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [81]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [82]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [83]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])

    # Lowercase the text
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(word.lower() for word in x.split()))

    # Remove URLs
    data["tweet"] = data["tweet"].apply(lambda x: re.sub(r'http\S+', '', x))

    # Remove punctuations
    data["tweet"] = data["tweet"].apply(remove_punctuations)

    # Remove digits
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)

    # Remove stopwords
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(word for word in x.split() if word not in sw))

    # Apply stemming
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(word) for word in x.split()))

    return data["tweet"]

In [84]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [85]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [90]:
txt = "awesome product. i  like it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'